#Data Extraction for SEMRYE project

Extract word pairs of rhymes in poems. Current for the collections of Shakespeare and Housman available in https://github.com/sravanareddy/rhymedata, courtesy of Sravana Reddy.


## Output Format

Output format would be string tuples: 

> ( word1:str, word2:str, rhyme pattern:str, rhyme letter:str, poetm title:str, section id:str, author:str)

**section id** starts from 0

For example,

**Input:**

>TITLE A Lover's Complaint

>RHYME a b a b b c c

>From off a hill whose concave womb reworded

>A plaintful story from a sistering vale,

>My spirits to attend this double voice reworded,

>And down I laid to list the sad-tuned tale;

>Ere long espied a fickle maid full pale, 

>Tearing of papers, breaking rings a-twain,

>Storming her world with sorrow's wind and rain.

> ...

For the full scheme of the data, see: https://github.com/sravanareddy/rhymedata

** Output:**

*(reworded, reworded, ababbcc, a, A Lover's Complaint, 1, Shakespeare )*

*(vale, tale, ababbcc, b, A Lover's Complaint, 1,  Shakespeare )*

*(vale, pale, ababbcc, b, A Lover's Complaint, 1, Shakespeare )*

*(tale, pale, ababbcc, b, A Lover's Complaint, 1, Shakespeare )*


...

** Note0** We may also want to keep a mapping **section_map**
- key: triple (poetm title <str>, section id <str>,  author <str>) 
- value: (section text, rhyme pattern)



**Note1** For now, if a rhyme involves multiple words, like the **b** rhyem above, which as 3 words *vale*, *tale*, and *pale*. We extract all the pairs and keep the partial order, i.e.  *(vale, tale)*, *(tale, pale)* and *(vale, pale)* for the example

**Note2** a special scheme used in the original data is 
> Sometimes, we use a shorthand for the rhyme scheme, like 

>RHYME a a *

>This denotes the rhyme scheme aabbccdd...

We elaborate such patterns in this case.




## Code Design

### Format of input data
The format of original data is quite regular
Each file dedicates to one author, which is specified in the first line

**poem structure**:
>an empty line

>title

>sections*

**section strucre**:

>empty line

>rhyme pattern

>empty line

>poem line*

> **UPDATE** It is NOT ture that each section starts with rhyme pattern, consecutive sections may share the same pattern, which is only specified before the 1st section.  See line 734 of Shakespear.txt for example

Poem lines are consecutive lines with content.

**Rhyme is at the last word of a poem line**


### Format of internal data

(discarded) **collections**: a list of **poem**, which is a tuple of (**poem title**, **list_of_section**)
(discarded) **list_of_section**: a list of **section**, each of which is a tuple (**section id**, **section text**, line_count)

#### Two outputs 

1. **word_pair_list**: a list of word pairs, each of the format **(word1 <str>, word2 <str>, rhyme pattern <str>, rhyme letter <str>, poetm title <str>, section id <str>, author <str> )**

2. **section_map**: a dictionary. key: (**poem title**, **section id**)  value: tuple(**section text** <list of list of string, i.e. splitted>, **rhyme pattern** <string>)


### Functions
Looks like there is no need for internal states offered by class. So the code will be procedure-based.


def **poem extract (file_name)**:
    1. get the *author_name* string
    
    loops for finding the poem
        get the *poem title*
        main *section_count*
        
        loop for finding sections
            get the *rhyme string*
            get *section text* and line count
            add entry to *section_map*
            
            call **parse_rhyme_pattern**, get rhyme pattern
            call **pair_extraction**, get word pair tuple, insert to *word_pair_list*
            

    return  word_pair_list, section_map, author_name 


def **parse_rhyme(rhyme string, line_count)**:
    
    return  new_rhyme_str, index_pair_list

Format of index_pair_list: (index1, index2, rhyme_letter)


def **pair_extraction(index_pair_list, section_text)** :
    return raw_pair_list
    

    
Format of each pair in raw_pair_list:  (word1, word2, letter)



In [87]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import print_function   # try to be python3 compatible, but it does not always work.
import string
#import codecs #might be useful for dealing with poems in other languages


def poem_extract(poem_file):
    
    # two output data structure
    section_map = {}
    word_pair_list = []
    
    debug = False
    
    with open(poem_file, 'r') as f: # as it's pure ascii
    #with codecs.open(poem_file, 'rU','utf-8') as f:  #change to it when handling with non-ascii langauge...
        lines = f.readlines()
        author_line = lines[0].split()
        lines = lines[1:]
        
    assert author_line[0]== 'AUTHOR'
    author_str = " ".join(author_line[1:])
    
    seen_text =   False
    #seen_rhyme = False
    section_text = []
    section_count = 0
    rhyme, poem_title = None, None
    
    total_line_count, total_section_count, total_title_count = 0, 0, 0
    
    for line_id, line in enumerate(lines):
        
        #print ('line num', line_id+2, ':', line.strip() ) 
        
        if line.split()==[]:
            #seen_rhyme = False
            if seen_text:
                
                assert  len(section_text) > 0
                assert rhyme
                assert poem_title
                
                total_section_count += 1                
              
                section_line_count = len(section_text)
                rhyme_str, index_pair_list = parse_rhyme(rhyme, section_line_count) # parse rhyme pattern
                
                section_map[(poem_title, section_count)] = (section_text, rhyme_str) # update section_map
                
                raw_pair_list = pair_extraction(index_pair_list, section_text) # extract word pairs in the rhyme
                
                extended_pair_list = [ (word1 , word2 , rhyme_str , rhyme_letter , poem_title , section_count , author_str ) for word1, word2, rhyme_letter in raw_pair_list ]
                word_pair_list.extend(extended_pair_list) # update word_pair_list
        
                if debug:
                    for word1, word2, rhyme_letter in raw_pair_list:
                        print ('record:', word1, word2, rhyme_letter)
        
                section_count += 1
                section_text = []
                section_line_count = 0
                
                # this logic of following two section does not hold, as line 724 in shakespear, where two consecutive sections with only one rhyme specified
                #rhyme = None
                #seen_rhyme = False
                seen_text = False
                
                
                
        
        elif line.split()[0] == 'TITLE':
            poem_title = " ".join(line.split()[1:])
            #seen_title = True           
            total_title_count += 1
            section_count = 0
            
        
        #elif  not seen_rhyme:  # the old logic does not hold, as line 724 in shakespear, where two consecutive sections with only one rhyme specified
            #assert line.split()[0] == 'RHYME'
        elif   line.split()[0] == 'RHYME':   
            rhyme = line.split()[1:]
            #seen_rhyme = True 
            if debug: print (rhyme)
        
        else:
            seen_text = True
            section_text.append(line.split())
            total_line_count += 1
    
    
    
    
    print ('\n\nDone! total line count/section count/title count=', total_line_count, total_section_count, total_title_count)
    
    return word_pair_list, section_map

        
            
#
# auxiliary function to compute the partial ordered pairs from a list of items
#
def get_partial_order_pair_list(list_of_item):
    partial_list = []
    if len(list_of_item)== 2:
        partial_list.append(tuple(list_of_item))

    else:
        for i, item in enumerate(list_of_item[:-1]):
            for j in range(i+1, len(list_of_item)):
                partial_list.append((item, list_of_item[j]))

    return partial_list            
            

def parse_rhyme(rhyme, section_line_count):
    if rhyme[-1] == '*':
        unit = rhyme[:-1]
        assert len(set(unit))==1
        base = ord(unit[0])
        
        assert section_line_count%len(unit) == 0
        
        repeat = (section_line_count- len(unit))/len(unit)
        rhyme =[]
        rhyme.extend(unit)
        for i in range(repeat):
            for j in range(len(unit)):
                rhyme.append(chr(base+i+1))
    
    assert len(rhyme) == section_line_count
    
    letter2index = {}
    for i, letter in enumerate(rhyme):
        letter2index.setdefault(letter, []).append(i)
    #return letter2index
    
    #Format of index_pair_list: (index1, index2, rhyme_letter)
    index_pair_list= []
    
    for rhyme_letter in letter2index:
        index_list = letter2index[rhyme_letter]
        partial_list = get_partial_order_pair_list(index_list)

        for pair in partial_list:
            index1, index2 = pair
            index_pair_list.append((index1, index2, rhyme_letter))


    index_pair_list.sort() 
    new_rhyme_str = ''.join(rhyme)

    return new_rhyme_str, index_pair_list


def remove_tail_punctuation(word):
    last = len(word)
    for i in range(last, 0, -1):
        #print (word[i-1: i])
        if word[i-1: i] not in string.punctuation:
            break
    return word[:i]


def pair_extraction(index_pair_list, section_text):
    
    debug = False
    # section_text are list of words (str)
    
    raw_pair_list = []
    #Format of each pair in raw_pair_list: (word1, word2, letter)
    
    list_of_last_word = [remove_tail_punctuation(sent[-1]) for sent in section_text]
    

    if debug: print ('list of last_word', list_of_last_word)
    
    for index1, index2, rhyme_letter in index_pair_list:
        raw_pair_list.append( (list_of_last_word[index1], list_of_last_word[index2], rhyme_letter))
    
    
    return raw_pair_list

    
    


In [66]:
#
# UNIT TEST for parse_rhyme()
#

r=get_partial_order_pair_list([1, 3, 4])
print ('sample partial list = ', r)


a=parse_rhyme('a b a b b c c'.split(), 7)
b=parse_rhyme('a a *'.split(), 8)
print (a, b)

sample partial list =  [(1, 3), (1, 4), (3, 4)]
('ababbcc', [(0, 2, 'a'), (1, 3, 'b'), (1, 4, 'b'), (3, 4, 'b'), (5, 6, 'c')]) ('aabbccdd', [(0, 1, 'a'), (2, 3, 'b'), (4, 5, 'c'), (6, 7, 'd')])


In [51]:
#
# UNIT TEST for pair_extraction() 
#

section_text = '''From off a hill whose concave womb reworded
A plaintful story from a sistering vale,
My spirits to attend this double voice accorded,
And down I laid to list the sad-tuned tale;
Ere long espied a fickle maid full pale, 
Tearing of papers, breaking rings a-twain,
Storming her world with sorrow's wind and rain.
'''

with open('tmp','w') as f:
    f.write(section_text)

with open('tmp','r') as f:
    text = f.readlines()
    section_text = [l.split() for l in text]
print (section_text)


new_rhyme_str, index_pair_list = parse_rhyme ('a b a b b c c'.split() ,7)
print ('index_pair_list:', index_pair_list)

raw_pair_list = pair_extraction(index_pair_list, section_text)

print ('\nRaw pair list:')
for i in raw_pair_list:
    print (i)

[['From', 'off', 'a', 'hill', 'whose', 'concave', 'womb', 'reworded'], ['A', 'plaintful', 'story', 'from', 'a', 'sistering', 'vale,'], ['My', 'spirits', 'to', 'attend', 'this', 'double', 'voice', 'accorded,'], ['And', 'down', 'I', 'laid', 'to', 'list', 'the', 'sad-tuned', 'tale;'], ['Ere', 'long', 'espied', 'a', 'fickle', 'maid', 'full', 'pale,'], ['Tearing', 'of', 'papers,', 'breaking', 'rings', 'a-twain,'], ['Storming', 'her', 'world', 'with', "sorrow's", 'wind', 'and', 'rain.']]
index_pair_list: [(0, 2, 'a'), (1, 3, 'b'), (1, 4, 'b'), (3, 4, 'b'), (5, 6, 'c')]
list of last_word ['reworded', 'vale,', 'accorded,', 'tale;', 'pale,', 'a-twain,', 'rain.']

Raw pair list:
('reworded', 'accorded,', 'a')
('vale,', 'tale;', 'b')
('vale,', 'pale,', 'b')
('tale;', 'pale,', 'b')
('a-twain,', 'rain.', 'c')


In [94]:
#
# Run the whole program for Shakespear
#
path_shake = '../data/shakespeare.txt'
word_pair_list_shakes, section_map_shakes =  poem_extract(path_shake)




Done! total line count/section count/title count= 5949 722 24


In [97]:
#
# Inspect the results for Shakespear
#

for i in word_pair_list_shakes [:20]:
    print (i)

k_list = section_map_shakes.keys()[8:10]
for k in k_list:
    print (k, ':', section_map_shakes[k])

('reworded', 'accorded', 'ababbcc', 'a', "A Lover's Complaint", 0, 'William Shakespeare')
('vale', 'tale', 'ababbcc', 'b', "A Lover's Complaint", 0, 'William Shakespeare')
('vale', 'pale', 'ababbcc', 'b', "A Lover's Complaint", 0, 'William Shakespeare')
('tale', 'pale', 'ababbcc', 'b', "A Lover's Complaint", 0, 'William Shakespeare')
('a-twain', 'rain', 'ababbcc', 'c', "A Lover's Complaint", 0, 'William Shakespeare')
('straw', 'saw', 'ababbcc', 'a', "A Lover's Complaint", 1, 'William Shakespeare')
('sun', 'done', 'ababbcc', 'b', "A Lover's Complaint", 1, 'William Shakespeare')
('sun', 'begun', 'ababbcc', 'b', "A Lover's Complaint", 1, 'William Shakespeare')
('done', 'begun', 'ababbcc', 'b', "A Lover's Complaint", 1, 'William Shakespeare')
('rage', 'age', 'ababbcc', 'c', "A Lover's Complaint", 1, 'William Shakespeare')
('eyne', 'brine', 'ababbcc', 'a', "A Lover's Complaint", 2, 'William Shakespeare')
('characters', 'tears', 'ababbcc', 'b', "A Lover's Complaint", 2, 'William Shakespeare'

In [95]:
#
# Run the whole program Housman
#

path_shake = '../data/housman.txt'
word_pair_list_housman, section_map_housman =  poem_extract(path_shake)




Done! total line count/section count/title count= 3215 659 177


In [103]:
# shakespeare writes way more sections per poem and more lines per sections...
print ('shakespeare:',722/24.0, 5949/722.0)
print ('housman',659/177.0, 3215/659.0)

shakespeare: 30.0833333333 8.23961218837
housman 3.72316384181 4.87860394537
